In [2]:
import sys
sys.executable

'c:\\users\\jonathan\\python_env\\nba_stat\\scripts\\python.exe'

In [37]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import NearestNeighbors
import graphviz 
import itertools

In [22]:
def remove_slash_name(_df):
    df = _df.copy()
    df['Player'] = df['Player'].apply(lambda row: row.split('\\')[0])
    df['Player'] = df['Player'].apply(lambda row: row.split('*')[0])
    
    return df

def remove_rk(_df):
    df = _df.copy()
    return df.drop(['Rk'], axis=1)

def remove_team(_df):
    df = _df.copy()
    return df.drop(['Tm'], axis=1)

def remove_age(_df):
    df = _df.copy()
    return df.drop(['Age'], axis=1)

def remove_game(_df):
    df = _df.copy()
    return df.drop(['G'], axis=1)   

def remove_game_started(_df):
    df = _df.copy()
    return df.drop(['GS'], axis=1)   

def remove_min(_df):
    df = _df.copy()
    return df.drop(['MP'], axis=1)    

def extract_name_position(_df):
    df = _df.copy()
    return _df.drop(['Pos','Player'], axis=1), _df[['Player', 'Pos']]
    

def remove_nan(_df):
    df = _df.copy()
    return df.dropna(axis=1, how='all').fillna(0)

In [50]:
def get_alike_player(name):
    # we use pandas to load data directly from csv
    df = pd.read_csv('adv_stats_95-96.csv')

    # apply a couple of preprocessing function
    df = remove_rk(remove_slash_name(df))
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_jordan, _ = extract_name_position(df.loc[df['Player'] == name])
    df_jordan

    df = pd.read_csv('adv_stats_18-19.csv')

    df = remove_rk(remove_slash_name(df))
    df = remove_age(remove_team(df))
    df = remove_game(remove_min(df))

    pos_used = ['PG','SG','SF','PF','C']
    df = df.loc[df['Pos'].isin(pos_used)]
    df = remove_nan(df)

    df_current_player, df_current_names = extract_name_position(df)
    neigh = NearestNeighbors(5)
    neigh.fit(df_current_player.values)

    dst, ids = neigh.kneighbors(df_jordan.values)
    return df_current_names.iloc[ids[0]]

In [54]:
player_names = ['Michael Jordan', 'Charles Barkley', 'Shawn Bradley', 'Muggsy Bogues', 'Patrick Ewing', 'Larry Johnson']
for p in player_names:
    print('Equivalent current player to {}'.format(p))
    print(get_alike_player(p))
    print()

Equivalent current player to Michael Jordan
             Player Pos
240     Paul George  SF
159   Stephen Curry  PG
191    Kevin Durant  PF
397   Kawhi Leonard  SF
401  Damian Lillard  PG

Equivalent current player to Charles Barkley
                 Player Pos
647  Karl-Anthony Towns   C
509        Jusuf Nurkic   C
574    Domantas Sabonis   C
661      Nikola Vucevic   C
161       Anthony Davis   C

Equivalent current player to Shawn Bradley
             Player Pos
36    Mohamed Bamba   C
258  JaMychal Green  PF
116  Wendell Carter   C
706     Ivica Zubac   C
448    JaVale McGee   C

Equivalent current player to Muggsy Bogues
                  Player Pos
170  Matthew Dellavedova  PG
634       Milos Teodosic  PG
225          Tim Frazier  PG
172  Matthew Dellavedova  PG
224          Tim Frazier  PG

Equivalent current player to Patrick Ewing
                Player Pos
655  Jonas Valanciunas   C
315        Serge Ibaka   C
554      Julius Randle  PF
656  Jonas Valanciunas   C
692     Chris